In [6]:
import os
import cv2
import numpy as np
from tqdm import tqdm
# Just for theme
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 200

In [2]:
DATADIR = 'C:\\Users\\home\\Desktop\\nirva-agni bscode\\Fire Dataset PCD\\train' # training path
CATEGORIES = ['neutral','fire','smoke']

In [3]:
IMG_SIZE = 64
def create_training_data():
    training_data = []
    for category in CATEGORIES:

        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)  # converting corresponding category

        for img in tqdm(os.listdir(path)):  # lặp qua từng ảnh một
            try:
                img_array = cv2.imread(os.path.join(path,img))  # chuyển đổi qua mảng
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # đặt lại size ảnh
                training_data.append([new_array, class_num])  # thêm vào tập training_data
            except Exception as e:  # giữ dữ liệu luôn sạch
                pass

    return training_data

In [4]:
training_data = create_training_data()

100%|██████████| 1380/1380 [00:24<00:00, 56.40it/s]


In [5]:
import random
print(len(training_data))
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

5144
1
1
0
2
0
2
0
0
2
0


In [6]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
X=np.array(X)
Y=np.array(Y)

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.2, random_state=42,shuffle=True)

In [8]:
print(x_test.shape,x_train.shape)

(1029, 64, 64, 3) (4115, 64, 64, 3)


In [12]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:]))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=3, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
history = model.fit(x_train, y_train, batch_size=64, epochs=50,validation_data=(x_test, y_test))

Epoch 1/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 26s 151ms/step - accuracy: 0.4873 - loss: 0.9906 - val_accuracy: 0.6035 - val_loss: 0.8564
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 120ms/step - accuracy: 0.6106 - loss: 0.8535 - val_accuracy: 0.6774 - val_loss: 0.7697
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 116ms/step - accuracy: 0.6714 - loss: 0.7593 - val_accuracy: 0.7221 - val_loss: 0.6608
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 117ms/step - accuracy: 0.7043 - loss: 0.6991 - val_accuracy: 0.7425 - val_loss: 0.6210
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 116ms/step - accuracy: 0.7165 - loss: 0.6552 - val_accuracy: 0.7648 - val_loss: 0.5769
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 117ms/step - accuracy: 0.7451 - loss: 0.6003 - val_accuracy: 0.7794 - val_loss: 0.5536
Epoch 7/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 116ms/step - accuracy: 0.7641 - loss: 0.5864 - val_accuracy: 0.7804 - val_loss: 0.5310
Epoch 8/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 119ms/step - accuracy: 0.7871 - loss: 0.5274 - val_acc

In [14]:

input_shape = model.input_shape
print("Expected input shape:", input_shape)

Expected input shape: (None, 64, 64, 3)


In [15]:
import numpy as np
input_image = np.random.random((64, 64, 3))  # Example input

# Normalize input image if needed (depends on how the model was trained)
input_image = input_image / 255.0

# Add batch dimension
input_image = np.expand_dims(input_image, axis=0)

# Make predictions
preds = model.predict(input_image)

# Process the predictions
print("Predictions:", preds)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
Predictions: [[0.8026875  0.1692112  0.02810128]]


In [16]:
model.save('C:/Users/home/Desktop/nirva-agni bscode/v1.h5')


In [12]:
input_image = np.expand_dims(X[6], axis=0)

# Make predictions
preds = model.predict(input_image)

# Process the predictions
print("Predictions:", preds)
l=np.argmax(model.predict(input_image))
l

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
Predictions: [[5.8820564e-01 1.9437496e-07 4.1179410e-01]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


0